<a href="https://colab.research.google.com/github/sasansharifipour/Spark_Class/blob/main/WORD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [167]:
!pip install gdown
!pip install parsivar

!gdown https://drive.google.com/u/0/uc?id=1N3hrFBDdOKyFv43LjSe5ynTdu_E0wOfP
!gdown https://drive.google.com/u/0/uc?id=1Cr3WJteka1SPQxi3-7swPobVmkwhwRp2
!gdown https://drive.google.com/u/0/uc?id=1zUwtMBlAfxlP69T_-Ldmb9DW3WzdfnPa
!gdown https://drive.google.com/u/0/uc?id=13gL9tfS3vwesBIcLtBnlSS3fTlZkWbDQ

Downloading...
From: https://drive.google.com/u/0/uc?id=1N3hrFBDdOKyFv43LjSe5ynTdu_E0wOfP
To: /content/train_set.csv
46.9MB [00:00, 178MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1Cr3WJteka1SPQxi3-7swPobVmkwhwRp2
To: /content/test_set.csv
12.1MB [00:00, 105MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1zUwtMBlAfxlP69T_-Ldmb9DW3WzdfnPa
To: /content/vectors.txt
118MB [00:00, 153MB/s] 
Downloading...
From: https://drive.google.com/u/0/uc?id=13gL9tfS3vwesBIcLtBnlSS3fTlZkWbDQ
To: /content/StopWords.csv
100% 3.27k/3.27k [00:00<00:00, 5.69MB/s]


In [6]:
import csv
import string
from bs4 import BeautifulSoup
import numpy as np
import re
import sys
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import csv

def prepare_output(text_Ids, y_pred):

  header = ['app_id', 'label']
  f = open('prediction.csv', 'w')
  writer = csv.writer(f)
  writer.writerow(header)

  i = 0
  while i < len(text_Ids):
    data = [text_Ids[i], y_pred[i]]
    writer.writerow(data)
    i += 1
  
  f.close()

def remove_emoji(string):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF" 
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U0001F917-\U0001F9D0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)

def replace_arabic_persian_alphabet(str):
  str = str.replace('ي','ی')
  str = str.replace('ك','ک')
  return str

def remove_html_tags(str):
  soup = BeautifulSoup(str, 'lxml')
  return soup.get_text()

def remove_half_space(str):
  return str.replace('\u200c', ' ')

def remove_special_symbols(str):
  str = str.replace(u'\xa0', u' ')
  str = str.replace(u'\u200f', u' ')
  str = str.replace(u'\n', u' ')
  str = str.replace(u'(', u' ')
  str = str.replace(u')', u' ')
  str = str.replace(u'.', u' ')
  str = str.replace(u'-', u' ')
  str = str.replace(u'،', u' ')
  str = str.replace(u'=', u' ')
  str = str.replace(u',', u' ')
  str = str.replace(u'?', u' ')
  str = str.replace(u'!', u' ')
  str = str.replace(u'؟', u' ')
  str = str.replace(u':', u' ')
  return str

def remove_white_space(str):
  return re.sub(' +', ' ', str)

def preprocess_text(str):
  txt = replace_arabic_persian_alphabet(str.lower())
  txt = remove_html_tags(txt)
  txt = remove_half_space(txt)
  txt = remove_special_symbols(txt)
  txt = remove_emoji(txt)
  txt = remove_white_space(txt)
  #txt = remove_punctuation_char(txt)
  return txt
    
def load_train_data():
  with open('train_set.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    X = []
    Y = []
    for row in csv_reader:
      if line_count == 0:
        line_count += 1
      else:
        txt = preprocess_text(row[1])
        X.append(txt)
        Y.append(int(row[2]))
        line_count += 1
  return X, Y

def load_test_data():
  with open('test_set.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    X = []
    X_IDs = []
    for row in csv_reader:
      if line_count == 0:
        line_count += 1
      else:
        X_IDs.append(row[0])
        X.append(preprocess_text(row[1]))
        line_count += 1
  return X,X_IDs

def prepare_train_data(train_data, stop_words):
  max_sentence_len = 8000
  max_cnt = 0
  train_len = len(train_data)
  train_x = np.zeros([train_len, max_sentence_len], dtype=np.int32)

  for i in range(train_len):
    train_x[i,:], cnt = get_sentence_embedding(train_X[i], stop_words, max_sentence_len)
    if (cnt > max_cnt):
      max_cnt = cnt
  
  train_x = train_x[:,:max_cnt]
  return train_x, max_cnt

def prepare_test_data(test_data, stop_words, max_sentence_len):
  test_len = len(test_data)
  test_x = np.zeros([test_len, max_sentence_len], dtype=np.int32)

  for i in range(test_len):
    test_x[i,:], cnt = get_sentence_embedding(test_X[i], stop_words, max_sentence_len)

  return test_x

def get_sentence_embedding(sentence, stopwords, max_sentence_len):
  words = sentence.split()
  sentence_embeddings = np.zeros(max_sentence_len)
  cnt = 0
  for word in words:
    if (word not in stop_words):
      word_info = word_model.wv.vocab.get(word)
      if (word_info != None):
        sentence_embeddings[cnt] = word_info.index
        cnt += 1
      if (cnt == max_sentence_len):
        break
  return sentence_embeddings, cnt

def load_stop_words():
  with open('StopWords.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    X = []
    for row in csv_reader:
        X.append(replace_arabic_persian_alphabet(row[0]))

  return X

glove_path = 'vectors.txt'
glove_filename = 'vectors'

word2vec_output_file = glove_filename+'.word2vec'
glove2word2vec(glove_path, word2vec_output_file)
word_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [7]:
train_X, train_Y = load_train_data()
test_X, test_IDs = load_test_data()
stop_words = load_stop_words()

In [8]:
train_x, max_sentence_len = prepare_train_data(train_X, stop_words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:140: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


2005

In [ ]:
from keras.utils import np_utils
import tensorflow as tf

pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape

train_x, max_sentence_len = prepare_train_data(train_X, stop_words)
test_x = prepare_test_data(test_X, stop_words, max_sentence_len)

train_y = np_utils.to_categorical(train_Y)

lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid')
])
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

NUM_EPOCHS = 10
history = lstm_model.fit(x= train_x, y= train_y, epochs=NUM_EPOCHS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:140: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          12027400  
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 128)         58880     
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 64)          0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 64)                24832     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)               

In [156]:
lstm_model.save('my_model.h5')
test_y_pred = lstm_model.predict(test_x)
y_pred = test_y_pred.argmax(1)
prepare_output(test_IDs, y_pred)

In [112]:
from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file

print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=10))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...


In [43]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    v = self.word_vectors.get_vector('در')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [44]:
vectorizer = Word2VecVectorizer(model)

Loading in word vectors...
Finished loading in word vectors


In [45]:
Xtrain = vectorizer.fit_transform(train_X)
Ytrain = train_Y

Numer of samples with no words found: 2 / 37899


In [46]:
Xtest = vectorizer.transform(test_X)

Numer of samples with no words found: 0 / 10000


In [59]:
Xtrain[0].shape

(50,)

In [48]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200)

clf.fit(Xtrain, Ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
y_pred = clf.predict(Xtest)

In [52]:
prepare_output(test_IDs, y_pred)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

parameters = { 
    'C': [1.0, 10],
    'gamma': [1, 'auto', 'scale']
}

model = GridSearchCV(SVC(kernel='rbf'), parameters, cv=5, n_jobs=-1).fit(Xtrain, Ytrain)

In [65]:
embedding_vector_features=50

model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(3877, embedding_vector_features,input_length=150))
model.add(tf.keras.layers.LSTM(128,input_shape=([150, 50]),activation='relu',return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 50)           193850    
_________________________________________________________________
lstm (LSTM)                  (None, 150, 128)          91648     
_________________________________________________________________
dropout (Dropout)            (None, 150, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)               